In [1]:
# Import dependencies
import os

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

2025-05-18 10:48:12.211539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747558092.312830    3595 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747558092.342187    3595 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747558092.581713    3595 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747558092.581720    3595 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747558092.581722    3595 computation_placer.cc:177] computation placer alr


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 16 intra-op, 9 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1747558096.173353    3595 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1747558096.174354    3595 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping, filter_no_finding_class
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights, oversample_minority_classes
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = 48
resume_training = True
checkpoint_path = '/home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/checkpoints/cp-0047.keras'
set_name        = "DenseNetV2"
model_name      = f"{set_name}_2"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

train_ds = filter_no_finding_class(train_ds)
val_ds   = filter_no_finding_class(val_ds)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [3]:
steps_per_epoch  = int((104166 or int(count_dataset_size(train_ds, None))) / TF_BATCH_SIZE)
validation_steps = int((15431 or int(count_dataset_size(val_ds, None))) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryFocalCrossentropy(gamma=2.0, from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max", save_weights_only=False)

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max", save_weights_only=False)

2025-05-18 10:49:58 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/train_metrics.csv
2025-05-18 10:49:58 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/val_metrics.csv
2025-05-18 10:49:59 - INFO - Cleaned training metrics file, kept 38211 records before epoch 48
2025-05-18 10:49:59 - INFO - Cleaned validation metrics file, kept 47 records before epoch 48
2025-05-18 10:49:59 - INFO - Found 38211 existing training records
2025-05-18 10:49:59 - INFO - Found 47 existing validation records


In [5]:
# Model Training
model          = build_densenet121(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-05-18 10:50:00 - INFO - Resuming training from checkpoint: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/checkpoints/cp-0047.keras
2025-05-18 10:50:01 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/train_metrics.csv
2025-05-18 10:50:01 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_2/val_metrics.csv
2025-05-18 10:50:01 - INFO - Cleaned training metrics file, kept 38211 records before epoch 48
2025-05-18 10:50:01 - INFO - Cleaned validation metrics file, kept 47 records before epoch 48
2025-05-18 10:50:01 - INFO - Found 38211 existing training records
2025-05-18 10:50:01 - INFO - Found 47 existing validation records
2025-05-18 10:50:01 - INFO - 
=== Training Started ===

2025-05-18 10:50:01 - INFO - Batch Size: 128
2025-05-18 10:50:01 - INFO - Optimizer: Adam
2025-05-18 10:50:01 - INFO - 

2025-05-18 10:50:0

Epoch 48/100


I0000 00:00:1747558221.179538    4168 service.cc:152] XLA service 0x715e2c006420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747558221.179580    4168 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1747558226.343032    4168 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747558282.612916    4168 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.9485 - auc: 0.9421 - f1_score: 0.5843 - loss: 0.0467 - precision: 0.7645 - recall: 0.4945  

2025-05-18 10:54:39 - INFO - 
=== Epoch 48 Summary ===
2025-05-18 10:54:39 - INFO - Time: 278.16s
2025-05-18 10:54:39 - INFO - Training   - accuracy: 0.9490 - auc: 0.9432 - f1_score: 0.5882 - loss: 0.0463 - precision: 0.7631 - recall: 0.4997 - learning_rate: 0.0000
2025-05-18 10:54:39 - INFO - Validation - accuracy: 0.9311 - auc: 0.7483 - f1_score: 0.2446 - loss: 0.0851 - precision: 0.3216 - recall: 0.2205
2025-05-18 10:54:39 - INFO - ==================================================



...

Epoch 74/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.9558 - auc: 0.9592 - f1_score: 0.6611 - loss: 0.0428 - precision: 0.7980 - recall: 0.5792  

2025-05-18 12:22:07 - INFO - 
=== Epoch 74 Summary ===
2025-05-18 12:22:07 - INFO - Time: 198.93s
2025-05-18 12:22:07 - INFO - Training   - accuracy: 0.9562 - auc: 0.9595 - f1_score: 0.6636 - loss: 0.0426 - precision: 0.7994 - recall: 0.5824 - learning_rate: 0.0000
2025-05-18 12:22:07 - INFO - Validation - accuracy: 0.9313 - auc: 0.7422 - f1_score: 0.2527 - loss: 0.0903 - precision: 0.3268 - recall: 0.2262
2025-05-18 12:22:07 - INFO - ==================================================



813/813 ━━━━━━━━━━━━━━━━━━━━ 199s 245ms/step - accuracy: 0.9558 - auc: 0.9592 - f1_score: 0.6611 - loss: 0.0428 - precision: 0.7980 - recall: 0.5792 - val_accuracy: 0.9313 - val_auc: 0.7422 - val_f1_score: 0.2527 - val_loss: 0.0903 - val_precision: 0.3268 - val_recall: 0.2262 - learning_rate: 1.0000e-06


2025-05-18 12:22:07 - INFO - 
=== Training Completed! ===

2025-05-18 12:22:07 - INFO - Final Metrics: accuracy: 0.9562 - auc: 0.9595 - f1_score: 0.6636 - loss: 0.0426 - precision: 0.7994 - recall: 0.5824 - val_accuracy: 0.9313 - val_auc: 0.7422 - val_f1_score: 0.2527 - val_loss: 0.0903 - val_precision: 0.3268 - val_recall: 0.2262



Epoch 74: early stopping
Restoring model weights from the end of the best epoch: 59.
